In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings('ignore')

possible_n_vals = [10]

def run_multiple_neural_nets(n):
    """Load dataset for 'n', split, scale, train three MLPs of increasing complexity,
    and report validation and test accuracies."""

    X = np.load('Datasets/kryptonite-%s-X.npy' % (n))
    y = np.load('Datasets/kryptonite-%s-y.npy' % (n))

    # Split: 60% train, 20% val, 20% test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    # Define three neural network configurations (increasing complexity)
    models = [
        {
            'name': 'nn_simple',
            'params': {'hidden_layer_sizes': (16,), 'activation': 'relu', 'solver': 'adam', 'alpha': 1e-4, 'max_iter': 500, 'random_state': 42},
        },
        {
            'name': 'nn_medium',
            'params': {'hidden_layer_sizes': (64,), 'activation': 'relu', 'solver': 'adam', 'alpha': 1e-4, 'max_iter': 1000, 'random_state': 42},
        },
        {
            'name': 'nn_deep',
            'params': {'hidden_layer_sizes': (128, 64), 'activation': 'relu', 'solver': 'adam', 'alpha': 1e-4, 'max_iter': 1500, 'random_state': 42},
        },
    ]

    results = []
    for m in models:
        clf = MLPClassifier(**m['params'])
        print(f"Training {m['name']} with hidden layers={m['params']['hidden_layer_sizes']} ...")
        clf.fit(X_train, y_train)

        # Evaluate on train set
        y_train_pred = clf.predict(X_train)
        train_acc = accuracy_score(y_train, y_train_pred)

        # Evaluate on validation set
        y_val_pred = clf.predict(X_val)
        val_acc = accuracy_score(y_val, y_val_pred)

        # Evaluate on test set
        y_test_pred = clf.predict(X_test)
        test_acc = accuracy_score(y_test, y_test_pred)

        print(f"{m['name']}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}, Test Acc = {test_acc:.4f}")
        results.append({'name': m['name'], 'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc, 'model': clf})

    # Summary: best by validation accuracy
    best = max(results, key=lambda r: r['val_acc'])
    print('\nBest model by validation accuracy: {} (train_acc={:.4f}, val_acc={:.4f}, test_acc={:.4f})'.format(best['name'], best['train_acc'], best['val_acc'], best['test_acc']))
    return results

# Example usage (uncomment to run):
results = run_multiple_neural_nets(10)
run_multiple_neural_nets(12)
run_multiple_neural_nets(14)
run_multiple_neural_nets(16)
run_multiple_neural_nets(18)
run_multiple_neural_nets(20)
# ...inspect results to pick best model...


Training nn_simple with hidden layers=(16,) ...
nn_simple: Val Acc = 0.9560, Test Acc = 0.9483
Training nn_medium with hidden layers=(64,) ...
nn_simple: Val Acc = 0.9560, Test Acc = 0.9483
Training nn_medium with hidden layers=(64,) ...
nn_medium: Val Acc = 0.9645, Test Acc = 0.9563
Training nn_deep with hidden layers=(128, 64) ...
nn_medium: Val Acc = 0.9645, Test Acc = 0.9563
Training nn_deep with hidden layers=(128, 64) ...
nn_deep: Val Acc = 0.9277, Test Acc = 0.9247

Best model by validation accuracy: nn_medium (val_acc=0.9645, test_acc=0.9563)
nn_deep: Val Acc = 0.9277, Test Acc = 0.9247

Best model by validation accuracy: nn_medium (val_acc=0.9645, test_acc=0.9563)
